In [762]:
class connected(object):
    def __init__(self, elementary_str, contractions, symbol):
        self.elementary_str = elementary_str #Stores list of N symbols for elementary field factors phi and dphi
        self.elementary = []
        self.contractions = contractions #Stores the indices with respect to which successive elementary factors are linked
                                #e.g., [[0,1],[1,0],[0,1]] for phi^i1 dphi_u1^i1 dphi_u1^i2 phi^i2, [[1,1]] for dphi_u^i dphi_u^i. For N 
                                #elementary fields, N-1 links.
        self.symbol = symbol
    def __str__(self):
        return 'elementary_str: ' + str(self.elementary_str) + '\n' \
            + 'contractions: ' + str(self.contractions) + '\n' \
            + 'symbol: ' + str(self.symbol) + '\n' \
            + 'massDim: ' + str(self.massDim()) + '\n' \
            + 'LORrank: ' + str(self.LORrank()) + '\n' \
            + 'ONrank: ' + str(self.ONrank()) 
    
    #Define get and set functions
    def get_elementary_str(self):
        return self.elementary_str
    def get_elementary(self):
        return self.elementary
    def get_contractions(self):
        return self.contractions
    def get_symbol(self):
        return self.symbol
    
    def set_elementary_str(self, elementary_str):
        self.elementary_str = elementary_str
    def set_elementary(self): #should run this to set list of constituent fields
        for symbol in self.elementary_str:
            if symbol == 'P':
                field = connected(['P'],[], 'P')
                self.elementary.append(field)
            elif symbol == 'DP':
                field_deriv = connected(['DP'],[], 'DP')
                self.elementary.append(field_deriv)
            else:
                print('Error: Symbol must be P or DP.')
    def set_contractions(self, contractions):
        self.contractions = contractions
    def set_symbol(self, symbol):
        self.symbol = symbol
    
    
    
    def massDim(self):
        
        counter = 0
        
        for field in self.get_elementary():
            if field.get_symbol() == 'P':
                counter = counter + 1
            elif field.get_symbol() == 'DP':
                counter = counter + 2
        
        return counter                       

    def LORrank(self):
        '''
        To get LORrank Count number of Lorentz contractions (counter) in contractions. Multiply by 2. Subtract this from 
        the number of Lorentz indices appearing in elementary - namely, the number of 'DP' fields. 
        '''
        counter1 = 0
        for pair in self.get_contractions():
            if pair[0] == 1:
                counter1 = counter1 + 1
            else:
                pass
            
        counter2 = 0
        for field in self.get_elementary():
            if field.get_symbol() == 'DP':
                counter2 = counter2 + 1
            else:
                pass     
        #print("counter1: " + str(counter1))
        #print("counter2: " + str(counter2))
        
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def ONrank(self):
        
        counter1 = 0
        
        #Number of contractions is the number of time a '1' appears in the second index of pairs in contractions
        for pair in self.get_contractions():
            if pair[1] == 1:
                counter1 = counter1 + 1
            else:
                pass
        
        #Number of ON indices is just the number of fields in self.elementary
        counter2 = len(self.get_elementary_str())
            
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def contract(self, other, contraction, symbol_prod):
        '''
        other: elementary field, P or DP
        contraction: pair of integers - [1,0] for contraction in LOR, [0,1] for contraction in ON, [1,1] for both
        symbol_prod: string
        '''
        
        #print("contraction: " + str(contraction))
        
        if contraction == [1,0]:
            contractions_prod = self.get_contractions() + [[1,0]]
        elif contraction == [0,1]:
            contractions_prod = self.get_contractions() + [[0,1]]
        elif contraction == [1,1]:
            contractions_prod = self.get_contractions() + [[1,1]]
        else:
            print("Error: contractions must be [1,0], [0,1], or [1,1]")
            
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        prod = connected(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        return prod
    
    def generate_connected(self, d):
        '''
        INPUT
        d: non-negative integer, mass dimension up to which to generate connected terms
        
        OUTPUT
        terms: list of connected terms of mass dimension no larger than d.  
        '''
        
        terms = []
        
        print("in generate_connected")
        
        if d >= 2:
            print("in d>=2")
            field1 = connected(['P'],[],'P')
            field2 = connected(['P'],[],'P')
            prod = connected.contract(field1, field2, [0,1], 'PP')
            terms.append(prod)
        if d >= 4:
            deriv1 = connected(['DP'],[],'DP')
            deriv2 = connected(['DP'],[],'DP')
            prod = connected.contract(deriv1, deriv2, [1,1], 'DPDP')
            terms.append(prod)
        
        field = connected(['P'],[],'P')
        deriv = connected(['DP'],[],'DP')
        
        if d < 4:
            return terms
    
        
        root = connected.contract(field, deriv, [0,1], 'PDP')
        
        while root.massDim() <= d:
            #print("")
            #print("root: ")
            #print(root)
            #print("")
            if root.LORrank() != 0:
                #print("in root.LORrank()!=0")
                deriv = connected(['DP'],[],'DP')
                root = connected.contract(root, deriv, [1,0], '')
                #print("")
                #print("root: ")
                #print(root)
                #print("")
            
            if root.ONrank() != 0:
                #print("in root.ONrank()!=0")
                field = connected(['P'],[],'P')
                deriv = connected(['DP'],[],'DP')
                
                new = connected.contract(root, field, [0,1], '')
                if new.massDim() <= d and new.LORrank() == 0 and new.ONrank() == 0:
                    #print("in new term added")
                    terms.append(new)
                
                root = connected.contract(root, deriv, [0,1], '')
                #print("")
                #print("root: ")
                #print(root)
                #print("")    
        
        return terms

    def generate(self, d):
        '''
        return list of all terms with mass dimension <= d
        '''
        terms = []
        if d < 2:
            terms = terms + [1]
        elif d == 2:
            new = generate_connected(2)
            terms = terms + new
        else:
            connected_list = generate_connected(d)
            max_term = connected_list[-1]
            d_next = connected_list[-2].massDim()
            
            connected_list_next = generate_connected(d_next)
            
            for term in connected_list_next:
                
                trial = connected.contract(max_term, term)
                
                if trial.massDim() <= d:
                    terms.append(trial)
                else:
                    pass
            
        
        
        return terms
        
        '''
        terms = [] #determine number of factors of
        
        connected_terms = self.generate_connected(d)
        connected_massDims = []
        len_connected = len(connected_terms)
        
        #Add connected terms to list of terms.
        #Build list of mass dimensions for each connected term. 
        for term in connected_terms:
            terms.append(term)
            connected_massDims.append(term.massDim())
            print(len(terms))
            
        print(connected_terms)
        
        exponents = [0]*len_connected   #initialize list containing number of powers of each connected term
        '''
        
        
        '''
        #procedure: 
        
        helper function: 
        def make_term(exponents):
            from list of exponents, constructs disconnected terms that can be written as field (here, "connected") objects
        
        loop through different values of exponents list
        {
        
        total_massDim = 0
        
        for i in range(len(connected_terms)):
            total_massDim += exponents[i]*connected_massDims[i]
            
        if total_massDim <= d:
            term = make_term(exponents)
            terms.append(term)
        }
        
        #how to do loop through exponents?
        
            
        
        '''
        
        
            
             
            
        
        return terms
        
    
            
    
    
    
   

In [759]:
 test = A.generate_connected(2)

in generate_connected
in d>=2


In [754]:
A = connected(['P','DP','DP','P'], [[0,1],[1,0],[0,1]], 'A')

In [761]:
print(test[0])

elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank: 0


In [756]:
for i in A.generate(10):
    print(i)

in generate_connected
in d>=2
1
2
3
4
[<__main__.connected object at 0x10cb79160>, <__main__.connected object at 0x10cbe2d30>, <__main__.connected object at 0x10cbe2e48>, <__main__.connected object at 0x10cbe23c8>]
elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank: 0
elementary_str: ['DP', 'DP']
contractions: [[1, 1]]
symbol: DPDP
massDim: 4
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 10
LORrank: 0
ONrank: 0


In [740]:
[0]*len(A.generate(10))

1
2
3
4
[<__main__.connected object at 0x10cbcacc0>, <__main__.connected object at 0x10cbca4a8>, <__main__.connected object at 0x10cbca0f0>, <__main__.connected object at 0x10cbca780>]


[0, 0, 0, 0]

In [663]:
A.get_elementary()

In [664]:
for i in range(len(A.get_elementary())):
    print(A.get_elementary()[i].get_symbol())


P
DP
DP
P


In [665]:
A.LORrank()

0

In [666]:
A.ONrank()

0

In [667]:
A.massDim()

6

In [668]:
B = connected(['P','DP','DP'], [[0,1],[1,0]], 'B' )

In [669]:
B.ONrank()

1

In [670]:
B.set_elementary()

In [671]:
B.get_elementary()

In [672]:
B.LORrank()

0

In [673]:
C = connected(['P','DP','DP','DP'], [[0,1],[1,0],[0,1]], 'C' )

In [674]:
C.get_elementary()

[]

In [675]:
C.set_elementary()

In [676]:
C.get_elementary()

In [677]:
C.LORrank()

1

In [678]:
C.ONrank()

0

In [679]:
C.get_contractions()

[[0, 1], [1, 0], [0, 1]]

In [680]:
C.get_contractions() + [1,0]

[[0, 1], [1, 0], [0, 1], 1, 0]

In [681]:
C.get_contractions() + [[1,0]]

[[0, 1], [1, 0], [0, 1], [1, 0]]

In [682]:
DP = connected(['DP'], [], 'DP')

In [683]:
connected.contract(C, DP,[1,0], 'Prod')

In [684]:
Prod = connected.contract(C, DP,[1,0],'Prod')

In [685]:
Prod.get_contractions()

[[0, 1], [1, 0], [0, 1], [1, 0]]

In [686]:
Prod.ONrank()

1

In [687]:
Prod.LORrank()

0

In [688]:
[1,0] == [1,0]

True

In [689]:
[1,0] == [1,0,1]

False

In [690]:
Test = C.generate_connected(10)

In [691]:
len(Test)

4

In [692]:
Test[0].get_elementary_str()

['P', 'P']

In [693]:
Test[1].get_elementary_str()

['DP', 'DP']

In [694]:
Test[0].get_contractions()

[[0, 1]]

In [695]:
Test[1].get_contractions()

[[1, 1]]

In [696]:
for i in Test:
    print(i)

elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank: 0
elementary_str: ['DP', 'DP']
contractions: [[1, 1]]
symbol: DPDP
massDim: 4
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 10
LORrank: 0
ONrank: 0


In [697]:
for i in C.generate_connected(20):
    print(i)

elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank: 0
elementary_str: ['DP', 'DP']
contractions: [[1, 1]]
symbol: DPDP
massDim: 4
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 10
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'DP', 'DP', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 14
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'DP', 'DP', 'DP', 'DP', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 18
LORrank: 0
ONrank: 0


In [704]:
C.generate(10)

[]

In [706]:
print(C)

elementary_str: ['P', 'DP', 'DP', 'DP']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: C
massDim: 7
LORrank: 1
ONrank: 0


In [708]:
C.generate_connected(10)

In [714]:
for i in C.generate_connected(10):
    print(i) 

elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank: 0
elementary_str: ['DP', 'DP']
contractions: [[1, 1]]
symbol: DPDP
massDim: 4
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 10
LORrank: 0
ONrank: 0


In [727]:
C.generate(20)

[]